In [3]:
import pandas as pd
import numpy as np
import shap
import torch
import scipy.stats

In [2]:
BASE_FOLDER = './outputs/evaluation'
PREFIX = 'oc_2way_500k_steps'
TRAFFIC = 'custom-2way-single-intersection'

In [4]:
df_single_episode = pd.read_csv(f'{BASE_FOLDER}/{PREFIX}_1_episode_{TRAFFIC}.csv')
df_single_episode = df_single_episode.set_index('step')
df_single_episode = df_single_episode.drop(columns=['cumulative_reward'])

In [ ]:
df_single_episode

In [6]:
df_single_episode[['option', 'termination_prob', 'should_terminate', 'greedy_option']].head(20)

,option,termination_prob,should_terminate,greedy_option
step,,,,
5.0,1,0.730739,True,1
10.0,1,0.730543,True,1
15.0,1,0.500094,False,0
20.0,1,0.500022,False,0
25.0,1,0.504440,False,0
30.0,1,0.515264,False,0
35.0,1,0.539745,False,0
40.0,1,0.537115,False,0
45.0,1,0.504101,False,1


In [91]:
# Convert obs and probabilities to separate dataset
obs = []
prob = df_single_episode.termination_prob.to_list()
for step, item in enumerate(df_single_episode.obs.tolist()):
    item = [float(s.strip()) for s in item.split(',')]
    data = {
        'prob': prob[step],
        'min_green': item[4]
    }
    for i in range(0, 4):
        data[f'green_light_{i}']: item[i]
    for i in range(8):
        item_index = i + 5
        data[f'lane_{i}_density'] = item[item_index]
        item_index2 = i + 13
        data[f'lane_{i}_queue'] = item[item_index2]
    obs.append(data)        
df_probs_obs = pd.DataFrame(obs)

In [93]:
df_probs_obs.cov()['prob'].sort_values(ascending=False)

lane_3_queue      0.024193
lane_7_queue      0.024166
lane_7_density    0.012937
lane_3_density    0.012914
prob              0.010366
lane_6_queue      0.005213
lane_2_queue      0.004881
lane_6_density    0.000701
lane_2_density    0.000600
min_green        -0.001140
lane_0_density   -0.002175
lane_4_density   -0.002246
lane_4_queue     -0.007863
lane_0_queue     -0.007912
lane_1_density   -0.009330
lane_5_density   -0.009984
lane_1_queue     -0.016863
lane_5_queue     -0.018622
Name: prob, dtype: float64

In [101]:
results = []
columns = df_probs_obs.columns
for column in columns:
    if column != 'prob':
        correlation = df_probs_obs['prob'].corr(df_probs_obs[column])
        results.append({
            'test': f'Correlation between prob and {column}',
            'correlation': correlation
        })
pd_correlation = pd.DataFrame(results)

In [103]:
pd_correlation.sort_values('correlation', ascending=False)

,test,correlation
7,Correlation between prob and lane_3_density,0.694905
8,Correlation between prob and lane_3_queue,0.684624
16,Correlation between prob and lane_7_queue,0.680571
15,Correlation between prob and lane_7_density,0.672383
14,Correlation between prob and lane_6_queue,0.188351
6,Correlation between prob and lane_2_queue,0.178133
13,Correlation between prob and lane_6_density,0.067971
5,Correlation between prob and lane_2_density,0.051511
0,Correlation between prob and min_green,-0.024738
9,Correlation between prob and lane_4_density,-0.117620


In [ ]:
lanes = ['n_t_0', 'n_t_1', 'e_t_0', 'e_t_1', 's_t_0', 's_t_1', 'w_t_0', 'w_t_1']
          0          1         2       3         4         5     6        7